In [27]:
import pandas as pd
import numpy as np

yelp_reviews = pd.read_csv("/content/Tweets.csv", sep=";")

yelp_reviews.head(10)


,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...
5,negative,@VirginAmerica seriously would pay $30 a fligh...
6,NaN,NaN
7,positive,"@VirginAmerica yes, nearly every time I fly VX..."
8,neutral,@VirginAmerica Really missed a prime opportuni...
9,positive,"@virginamerica Well, I didn't…but NOW I DO! :-D"


Nous allons simplifier notre problème en convertissant les valeurs numériques des avis en valeurs catégorielles. Cela se fera en ajoutant une nouvelle colonne, reviews_score, à notre ensemble de données. Si l'avis de l'utilisateur a une valeur comprise entre 1 et 2 dans la colonne stars (qui évalue l'entreprise sur une échelle de 1 à 5), la colonne reviews_score aura une valeur de chaîne négative. Si la note est comprise entre 3 et 5 dans la colonne stars, la colonne reviews_score contiendra une valeur positive. Cela fait de notre problème un problème de classification binaire.


In [28]:
yelp_reviews.drop_duplicates(inplace=True)
yelp_reviews.drop([6739, 4112, 6189, 13479, 4672, 10120, 7504, 7421, 13351], inplace = True)
yelp_reviews.dropna(inplace  = True)
yelp_reviews.isna().sum()

airline_sentiment    0
text                 0
dtype: int64

In [29]:
yelp_reviews.head(10)

,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...
5,negative,@VirginAmerica seriously would pay $30 a fligh...
7,positive,"@VirginAmerica yes, nearly every time I fly VX..."
8,neutral,@VirginAmerica Really missed a prime opportuni...
9,positive,"@virginamerica Well, I didn't…but NOW I DO! :-D"
10,positive,"@VirginAmerica it was amazing, and arrived an ..."


**Installation de FastText**

In [30]:
!wget https://github.com/facebookresearch/fastText/archive/v0.1.0.zip

--2023-06-15 08:57:23--  https://github.com/facebookresearch/fastText/archive/v0.1.0.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/facebookresearch/fastText/zip/refs/tags/v0.1.0 [following]
--2023-06-15 08:57:23--  https://codeload.github.com/facebookresearch/fastText/zip/refs/tags/v0.1.0
Resolving codeload.github.com (codeload.github.com)... 140.82.113.9
Connecting to codeload.github.com (codeload.github.com)|140.82.113.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘v0.1.0.zip’

v0.1.0.zip              [ <=>                ]  92.06K  --.-KB/s    in 0.04s   

2023-06-15 08:57:24 (2.46 MB/s) - ‘v0.1.0.zip’ saved [94267]



L'étape suivante consiste à décompresser les modules FastText. Tapons simplement la commande suivante :

In [31]:
!unzip v0.1.0.zip

Archive:  v0.1.0.zip
431c9e2a9b5149369cc60fb9f5beba58dcf8ca17
   creating: fastText-0.1.0/
  inflating: fastText-0.1.0/.gitignore  
  inflating: fastText-0.1.0/CONTRIBUTING.md  
  inflating: fastText-0.1.0/LICENSE  
  inflating: fastText-0.1.0/Makefile  
  inflating: fastText-0.1.0/PATENTS  
  inflating: fastText-0.1.0/README.md  
  inflating: fastText-0.1.0/classification-example.sh  
  inflating: fastText-0.1.0/classification-results.sh  
  inflating: fastText-0.1.0/eval.py  
  inflating: fastText-0.1.0/get-wikimedia.sh  
  inflating: fastText-0.1.0/pretrained-vectors.md  
  inflating: fastText-0.1.0/quantization-example.sh  
  inflating: fastText-0.1.0/quantization-results.sh  
   creating: fastText-0.1.0/src/
  inflating: fastText-0.1.0/src/args.cc  
  inflating: fastText-0.1.0/src/args.h  
  inflating: fastText-0.1.0/src/dictionary.cc  
  inflating: fastText-0.1.0/src/dictionary.h  
  inflating: fastText-0.1.0/src/fasttext.cc  
  inflating: fastText-0.1.0/src/fasttext.h  
  inflat

Ensuite, nous devons naviguer jusqu'au répertoire où nous avons téléchargé FastText, puis exécuter la commande !make pour exécuter les binaires C++. Exécutez les étapes suivantes :

In [32]:
cd fastText-0.1.0

/content/fastText-0.1.0/fastText-0.1.0


In [33]:
!make

c++ -pthread -std=c++0x -O3 -funroll-loops -c src/args.cc
c++ -pthread -std=c++0x -O3 -funroll-loops -c src/dictionary.cc
c++ -pthread -std=c++0x -O3 -funroll-loops -c src/productquantizer.cc
c++ -pthread -std=c++0x -O3 -funroll-loops -c src/matrix.cc
c++ -pthread -std=c++0x -O3 -funroll-loops -c src/qmatrix.cc
c++ -pthread -std=c++0x -O3 -funroll-loops -c src/vector.cc
c++ -pthread -std=c++0x -O3 -funroll-loops -c src/model.cc
c++ -pthread -std=c++0x -O3 -funroll-loops -c src/utils.cc
c++ -pthread -std=c++0x -O3 -funroll-loops -c src/fasttext.cc
c++ -pthread -std=c++0x -O3 -funroll-loops args.o dictionary.o productquantizer.o matrix.o qmatrix.o vector.o model.o utils.o fasttext.o src/main.cc -o fasttext


Pour vérifier l'installation, exécuter:

In [34]:
!./fasttext

usage: fasttext <command> <args>

The commands supported by fasttext are:

  supervised              train a supervised classifier
  quantize                quantize a model to reduce the memory usage
  test                    evaluate a supervised classifier
  predict                 predict most likely labels
  predict-prob            predict most likely labels with probabilities
  skipgram                train a skipgram model
  cbow                    train a cbow model
  print-word-vectors      print word vectors given a trained model
  print-sentence-vectors  print sentence vectors given a trained model
  nn                      query for nearest neighbors
  analogies               query for analogies



**Classement du texte**

Avant de former des modèles FastText pour effectuer une classification de texte, il est pertinent de mentionner que FastText accepte les données dans un format spécial, qui est le suivant :

_label_tag This is sentence 1
_label_tag2 This is sentence 2.

Le script suivant filtre les colonnes **reviews_score** et **text** du jeu de données, puis préfixe __label__ avant toutes les valeurs de la colonne **reviews_score**.

De même, le **\n** et le **\t** sont remplacés par un **espace** dans la colonne de texte.

Enfin, la trame de données mise à jour est écrite sur le disque sous la forme de **yelp_reviews_updated.txt**.



In [35]:
import pandas as pd
from io import StringIO
import csv
# col = ['airline_sentiment', 'text']

# yelp_reviews = yelp_reviews[col]
yelp_reviews['airline_sentiment']=['__label__'+ s for s in yelp_reviews['airline_sentiment']]
yelp_reviews['text']= yelp_reviews['text'].replace('\n',' ', regex=True).replace('\t',' ', regex=True).replace('@',' ', regex=True)
yelp_reviews.to_csv(r'/content/yelp_reviews_updated.txt', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")

In [36]:
yelp_reviews.head()

,airline_sentiment,text
0,__label__neutral,VirginAmerica What dhepburn said.
1,__label__positive,VirginAmerica plus you've added commercials t...
2,__label__neutral,VirginAmerica I didn't today... Must mean I n...
3,__label__negative,VirginAmerica it's really aggressive to blast...
4,__label__negative,VirginAmerica and it's a really big bad thing...


L'étape suivante consiste à diviser nos données en ensembles d'apprentissage et de test

Les 40 000 premiers enregistrements sur 50 000, seront utilisées pour former les données
Les 10 000 derniers enregistrements) seront utilisées pour évaluer les performances de l'algorithme

In [37]:
!head -n 12000 "/content/yelp_reviews_updated.txt" > "/content/yelp_reviews_train.txt"
!tail -n 2393 "/content/yelp_reviews_updated.txt" > "/content/yelp_reviews_test.txt"

Une fois le script ci-dessus exécuté, le fichier **yelp_reviews_train.txt** sera généré, qui contient les données de formation.

De même, le fichier **yelp_reviews_test.txt** nouvellement généré contiendra les données de test.

Il est maintenant temps de former notre algorithme de classification de texte FastText.

Pour former l'algorithme, nous devons utiliser une commande **supervisée** et lui transmettre le fichier d'entrée. Le nom du modèle est spécifié après le mot-clé -output. Le script ci-dessus se traduira par un modèle de classification de texte formé appelé **model_yelp_reviews.bin**.

In [38]:
%%time
!./fasttext supervised -input "/content/yelp_reviews_train.txt" -output model_yelp_reviews

Read 0M words
Number of words:  25625
Number of labels: 3
Progress: 100.0%  words/sec/thread: 2175107  lr: 0.000000  loss: 0.614832  eta: 0h0m 
CPU times: user 33.8 ms, sys: 3.53 ms, total: 37.3 ms
Wall time: 2.02 s


In [39]:
!ls

args.o			   Makefile		    quantization-results.sh
classification-example.sh  matrix.o		    README.md
classification-results.sh  model.o		    src
CONTRIBUTING.md		   model_yelp_reviews.bin   tutorials
dictionary.o		   model_yelp_reviews.vec   utils.o
eval.py			   PATENTS		    vector.o
fasttext		   pretrained-vectors.md    wikifil.pl
fasttext.o		   productquantizer.o	    word-vector-example.sh
get-wikimedia.sh	   qmatrix.o
LICENSE			   quantization-example.sh


Nous pouvons voir le **model_yelp_reviews.bin** dans la liste de documents ci-dessus.

Enfin, pour tester le modèle, nous pouvons utiliser la commande **test**. nous devons spécifier le nom du modèle et le fichier de test après la commande de test, comme indiqué ci-dessous :

In [40]:
!./fasttext test model_yelp_reviews.bin "/content/yelp_reviews_test.txt"

N	2393
P@1	0.817
R@1	0.817
Number of examples: 2393


Ici, **P@1** fait référence à **la précision** et **R@1** au **Recall**. Nous pouvons voir que notre modèle atteint une précision et un Racall de **0,909**, ce qui est plutôt bon.

Essayons maintenant de nettoyer notre texte des ponctuations, des caractères spéciaux et de le convertir en minuscules pour améliorer l'uniformité du texte. Le script suivant nettoie la rame :

In [41]:
!cat "/content/yelp_reviews_train.txt" | sed -e "s/\([.\!?,’/()]\)/ \1 /g" | tr "[:upper:]" "[:lower:]" > "/content/yelp_reviews_train_clean.txt"

Et le script suivant nettoie l'ensemble de test

In [42]:
!cat "/content/yelp_reviews_test.txt" | sed -e "s/\([.\!?,’/()]\)/ \1 /g" | tr "[:upper:]" "[:lower:]" > "/content/yelp_reviews_test_clean.txt"

Maintenant, nous allons entraîner le modèle sur l'ensemble d'entraînement nettoyé :

In [24]:
%%time
!./fasttext supervised -input "/content/yelp_reviews_train_clean.txt" -output model_yelp_reviews

Read 0M words
Number of words:  15565
Number of labels: 3
Progress: 100.0%  words/sec/thread: 2642649  lr: 0.000000  loss: 0.604100  eta: 0h0m 
CPU times: user 27.5 ms, sys: 1.87 ms, total: 29.3 ms
Wall time: 1.32 s


Et enfin, nous utiliserons le modèle entraîné sur l'ensemble d'apprentissage nettoyé pour faire des prédictions sur l'ensemble de test nettoyé :

In [43]:
!./fasttext test model_yelp_reviews.bin "/content/yelp_reviews_test_clean.txt"

N	2393
P@1	0.807
R@1	0.807
Number of examples: 2393


Nous pouvons voir une légère augmentation de la précision et du rappel. Pour améliorer encore le modèle, nous pouvons augmenter les époques et le taux d'apprentissage du modèle. Le script suivant définit le nombre d'époques sur 30 et le taux d'apprentissage sur 0,5.

In [44]:
%%time
!./fasttext supervised -input "/content/yelp_reviews_train_clean.txt" -output model_yelp_reviews -epoch 30 -lr 0.5

Read 0M words
Number of words:  15565
Number of labels: 3
Progress: 100.0%  words/sec/thread: 2717613  lr: 0.000000  loss: 0.263495  eta: 0h0m 
CPU times: user 73.4 ms, sys: 3.78 ms, total: 77.2 ms
Wall time: 2.69 s


Nous pouvons essayer différents nombres et voir si vous pouvez obtenir de meilleurs résultats.

**Conclusion:**

Le modèle FastText s'est récemment révélé à la pointe de la technologie pour les incorporations de mots et les tâches de classification de texte sur de nombreux ensembles de données. Il est très facile à utiliser et rapide  par rapport aux autres modèles d'intégration de mots.
